### Steps to mount to a Azure storage account
The steps are adopted from https://www.mssqltips.com/sqlservertip/6931/mount-azure-data-lake-storage-gen2-account-databricks/

1. Register a new web application
    - Go to your ***Azure Portal***
    - **Home** > **App registrations**
    - Click + **New registrations**
    - Give your app a name and choose who can use the app. I chose ***Accounts in this organizational directory only (Microsoft only - Single tenant)***. Next, click the ***Register*** button.
    - Clicking the ***Register*** button will take you to the ***Overview*** page of your app. This page contains some valuable information that you need to record. Save the following information to a notepad. We will store them securely in a key vault in the **Step 2**.
      - Directory (tenant) ID
      - Application (client) ID
      - Go to the Certificates and Secrets section of the app and create a new client secret by clicking on the New client secret button. Copy the secret and save it to notepad
      
2. Store the values from web app in a **Azure Key Vault** using the steps below
    - Create a Azure Key vault resource if you don't already have one. There are multiple ways you can create one. One of those methods are listed below:
      - Go to your ***Azure Portal*** 
      - Search for ***key vault*** in the search bar at the top of the page
      - Click the ***+ Create*** link in the upper left of the page to create a new ***Key vault***
      - Fill out the form
        - ***Subscription***:  Choose the ***Subscription*** you want to use for the project.
        - ***Resource group***:  Choose the ***Resource group*** you want to use for the project.
        - ***Key vault name***:  Create a key vault name. The recommended naming convention is **\<resource type\>-\<application\>-\<environment\>-\<Azure Region\>** so  an example name is **kv-lakehousetalk-prod-northcentral**
        - ***Region***: The Azure region you want to use
        I went with the defaults for the other options. Your ***Azure Administrator*** will know what is best for your situation. At this point you can click ***Review + create*** to create the key vault. 
    - Add secrets to the key vault you created for ***Directory (tenant) ID***, ***Application (client) ID***, and ***Client Secret***. Perform the following steps to do so:
      - Go to the key vault that you created
      - On the left pane scroll down to the ***Objects*** section and click on ***Secrets***
      - Click ***Generate/Import*** in the upper left corner
      - Give your secret a name in the ***Name*** text box and the value of your secret in the ***Secret Value*** text box
      - Click on the ***Set activation date*** and ***Set expiration date*** check box. This will set the secret to be good for 2 years.
      - Make sure the secret is enabled then click ***Create***
      - Repeat the process for each secret
      - Recommended secret names
        - ***kv-clientid-\<environment>-\<azure region>*** for the client ID
        - ***kv-tenantid-\<environment>-\<azure region>*** for the tenant ID
        - ***kv-applicationid-\<environment>-\<azure region>*** for the applicationID
        - Additional Notes
          - Environment options are normally test, dev, qa, and prod. It depends on the development strategy of your oganization
          - Here is an example of what a key vault name could look like:  **kv-tenantid-dev-northcentralus**
          
3. Give the app ***Storage Blob Data Contributor*** access to the storage account
    - Go to your ***resource group*** for this project and open your ***storage account***
    - On the left pane click on ***Access Control (IAM)***
    - Click on ***+ Add*** then ***Add role assignment***
    - Select ***Storage Blob Data Contributor*** in the list of roles then click ***Next***. You can use the search box to help you find it.
    - Select ***User, group, or service principal*** then click ***+ Select members***
    - Search for the app you created in ***Step 1*** then click ***Select***
    - Click ***Review + Assign***
4. Create a scope in Databricks to connect to Key Vault
    - Build the URL using the following template:  https://\<DATABRICKS-INSTANCE\>#secrets/createScope
    - Paste the URL created in the previous step in the browser then press enter
    - Fill out the form that appears
      - Give it a meaningful scope name
      - Go to the properties section of the Key Vault
        - Copy the DNS Name (Vault URI) value and paste it in the DNS section
        - Copy the Resource ID value and paste it in the Resource ID section
5. Mount Databricks to your ADLS Gen 2 using the code below (talk about the variables in cell block one)

In [0]:
adlsAccountName = "<storage account name>"
adlsContainerName = "<storage container name>"
mountPoint = "<mount point name>"
databricksKeyVaultScope = "<DatabricksvKeyVault Scope>"
clientIDKVSecretName = "<Client ID Key Vault Secret Name>"
tenantIDKVSecretName = "<Tenant ID Key Vault Secret Name>"
applicationIDKVSecretName = "<Tenant ID Key Vault Secret Name>"

In [0]:
clientId = dbutils.secrets.get(scope=databricksKeyVaultScope,key=clientIDKVSecretName)
clientSecret = dbutils.secrets.get(scope=databricksKeyVaultScope,key=applicationIDKVSecretName)
tenantId = dbutils.secrets.get(scope=databricksKeyVaultScope,key=tenantIDKVSecretName)

endpoint = "https://login.microsoftonline.com/" + tenantId + "/oauth2/token"
source = "abfss://" + adlsContainerName + "@" + adlsAccountName + ".dfs.core.windows.net/"
 
# Connecting using Service Principal secrets and OAuth
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientId,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": endpoint}
 
# Mount ADLS Storage to DBFS only if the directory is not already mounted
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
    source = source,
    mount_point = mountPoint,
    extra_configs = configs)